
# 🩺 Détection de Pneumonie à partir de Radiographies (KNN)

Ce notebook implémente un modèle **K-Nearest Neighbors** pour classer des radiographies pulmonaires en deux classes :
- `NORMAL` : pas de signe de pneumonie
- `PNEUMONIA` : signes de pneumonie détectés

Le dataset est local, organisé dans les dossiers `train/`, `val/` et `test/`.

---


In [ ]:

import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from PIL import Image
from ipywidgets import interact, IntSlider


## ⚙️ Configuration

In [ ]:

# Paramètres globaux
IMAGE_SIZE = (128, 128)
DATASET_PATH = './datasets'  # à adapter si besoin
USE_PCA = False               # True pour activer la réduction de dimension
K_NEIGHBORS = 3               # nombre de voisins pour le modèle KNN


## 🗂️ Chargement et préparation des données

In [ ]:

def load_images_from_folder(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        if not filename.lower().endswith(('.jpeg', '.jpg', '.png')):
            continue
        path = os.path.join(folder, filename)
        try:
            img = Image.open(path).convert("L").resize(IMAGE_SIZE)
            img_array = np.asarray(img).flatten() / 255.0
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Erreur avec {path} : {e}")
    return images, labels

def load_dataset(base_path, subsets):
    X, y = [], []
    for subset in subsets:
        for label_str, label_val in [("NORMAL", 0), ("PNEUMONIA", 1)]:
            folder = os.path.join(base_path, subset, label_str)
            imgs, labels = load_images_from_folder(folder, label_val)
            X.extend(imgs)
            y.extend(labels)
    return np.array(X), np.array(y)


In [ ]:

print("🔄 Chargement des données...")
X_train, y_train = load_dataset(DATASET_PATH, ["train", "val"])
X_test, y_test = load_dataset(DATASET_PATH, ["test"])
print(f"✅ {len(X_train)} images d'entraînement, {len(X_test)} images de test")


## 🔍 Réduction de dimension (optionnelle avec PCA)

In [ ]:

if USE_PCA:
    print("Application de PCA pour réduction de dimension...")
    pca = PCA(n_components=100)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
else:
    print("PCA désactivée")


## 🧠 Entraînement du modèle KNN

In [ ]:

model = KNeighborsClassifier(n_neighbors=K_NEIGHBORS)
model.fit(X_train, y_train)
print(f"Modèle KNN entraîné avec k={K_NEIGHBORS}")


## 📊 Évaluation du modèle

In [ ]:

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["NORMAL", "PNEUMONIA"]))


## 🖼️ Visualisation interactive d'exemples

In [ ]:

def show_image(index):
    img = X_test[index].reshape(IMAGE_SIZE)
    true_label = "PNEUMONIA" if y_test[index] == 1 else "NORMAL"
    pred_label = "PNEUMONIA" if y_pred[index] == 1 else "NORMAL"
    color = "green" if y_test[index] == y_pred[index] else "red"

    plt.figure(figsize=(3, 3))
    plt.imshow(img, cmap='gray')
    plt.title(f"Vraie étiquette: {true_label}\nPrédiction: {pred_label}", color=color)
    plt.axis('off')
    plt.show()

interact(show_image, index=IntSlider(min=0, max=len(X_test)-1, step=1, value=0));
